### Read SLHA, SModelS output and store the data in a pandas DataFrame

In [1]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import pandas as pd
import glob,os
import numpy as np
from pandas import json_normalize
from importlib import util
import pyslha
import sys
sys.path.append(os.path.expanduser('~/smodels'))
from smodels.share.models.SMparticles import SMList
from smodels.base.model import Model
from smodels.tools.particlesLoader import getParticlesFromSLHA

pd.options.mode.chained_assignment = None #Disable copy warnings

In [2]:
BSMlist = getParticlesFromSLHA('./2mdm_example.slha')
model = Model(BSMparticles=BSMlist, SMparticles=SMList)
print(BSMlist)

[gha, gha~, ghz, ghz~, ghwp, ghwp~, ghwm, ghwm~, ghg, ghg~, zp, ghzp, ghzp~, sd, chi]


In [3]:
slhaFolder = '../data/slha_files'
smodelsFolder = '../data/smodels_results'
parametersSmodels = './smodels_parameters_2mdm.ini'

In [4]:
#Convert Experimental Results list to a dictionary
data = []
removeFromDict = ['topologies outside the grid',"missing topologies",
                  "missing topologies with displaced decays", 
                  'missing topologies with prompt decays',
                 "Asymmetric Branches","Outside Grid",
                 "Missed Topologies","Long Cascades",
                 "OutputStatus",
                #  "Total xsec for missing topologies (fb)", 
                #  "Total xsec for missing topologies with displaced decays (fb)",
                #  "Total xsec for missing topologies with prompt decays (fb)",
                #  "Total xsec for topologies outside the grid (fb)"
                ]

nfiles = 0
exptResData = []
for f in glob.glob(smodelsFolder+'/*.py'):
    # nfiles += 1
    # if nfiles > 5:
        # break
    spec = util.spec_from_file_location("smodelsOutput", f)
    smodelsOutput = util.module_from_spec(spec)
    spec.loader.exec_module(smodelsOutput)
    smodelsDict = smodelsOutput.smodelsOutput
    for rmKey in removeFromDict:
        if rmKey in smodelsDict:
            smodelsDict.pop(rmKey)
        slhaFile = f.replace('.py','')
    smodelsDict['filename'] = os.path.basename(slhaFile)
    if 'CombinedRes' in smodelsDict:
        smodelsDict['ExptRes'] += smodelsDict.pop('CombinedRes')
    if 'ExptRes' in smodelsDict:
        expList = sorted(smodelsDict.pop('ExptRes'), 
                            key=lambda pt: (pt['r'] is not None,pt['r']),
                            reverse=True)
        for iexp,exp in enumerate(expList):
            # newDict = {k : v for k,v in smodelsDict.items()}
            newDict = {'filename' : smodelsDict['filename']}
            newDict['ExptRes'] = iexp
            newDict.update({k : v for k,v in exp.items()})
            exptResData.append(newDict)
    else:
        newDict = {'filename' : smodelsDict['filename'], 'ExptRes' : None}
        exptResData.append(newDict)
    data.append(smodelsDict)


In [5]:
dataDF = pd.DataFrame(data)
dataDF.set_index('filename',inplace=True)
expDF = pd.DataFrame(exptResData)
expDF.set_index(['filename','ExptRes'],inplace=True)
smodelsDF = expDF.join(dataDF, how='outer')
# smodelsDF = expDF.join(dataDF, how='inner')
print(f'Number of data points: {len(smodelsDF.groupby(level=0))}')

Number of data points: 21000


In [6]:
#Get SLHA data:
slhaData = []
for f in smodelsDF.index.get_level_values('filename').unique():
    slhaFile = os.path.join(slhaFolder,f)
    slha = pyslha.readSLHAFile(slhaFile)
    massDict = dict([[str(key),abs(val)] for key,val in slha.blocks['MASS'].items() if key >= 52])
    extparDict = dict([[str(key),val] for key,val in slha.blocks['NPINPUTS'].items()])
    
    widthDict = dict([[str(key),val.totalwidth] for key,val in slha.decays.items() if key >= 52])
    BRsDict = {}
    for pdg,val in slha.decays.items():
        initialState = model.getParticlesWith(pdg=pdg)[0].label            
        BRsDict[initialState] = {}
        for dec in val.decays:
            if dec.br < 1e-7: continue            
            finalState = ','.join([model.getParticlesWith(pdg=pid)[0].label for pid in sorted(dec.ids)])
            BRsDict[initialState][finalState] = dec.br
    xsec8TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=8000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=8000)])
    xsec13TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=13000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=13000)])    
    slhaDict = {'filename' : f, 'mass' : massDict, 'width' : widthDict, 'extpar' : extparDict, 
                'xsec8TeV(fb)' : xsec8TeV, 'xsec13TeV(fb)' : xsec13TeV, 'BRs' : BRsDict}
    slhaData.append(slhaDict)

In [7]:
slhaDF = json_normalize(slhaData)
slhaDF.set_index(['filename'],inplace=True)
#Add total cross-sections:
xsecs13 = [x for x in list(slhaDF) if 'xsec13TeV' in x]
xsecs8 = [x for x in list(slhaDF) if 'xsec8TeV' in x]
slhaDF['totalxsec13TeV(fb)'] = slhaDF[xsecs13].sum(axis=1)
slhaDF['totalxsec8TeV(fb)'] = slhaDF[xsecs8].sum(axis=1)

In [8]:
#Merge with SModelS DataFrame
allDF = smodelsDF.join(slhaDF, how='outer')

In [9]:
print(f'Final number of data points: {len(allDF.groupby(level=0))}')
#print(dataDF2.columns.values.tolist()) #Print all columns names

Final number of data points: 21000


In [10]:
#Save DataFrame to pickle file:
allDF.to_pickle('../data/smodels_results_v3.pcl')